In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import chromadb
from sentence_transformers import SentenceTransformer
import re
from typing import Dict, List, Optional, Tuple

class ChatCUCEI:
    def __init__(self, base_model="microsoft/Phi-3-mini-4k-instruct", chroma_db_path="./chroma_db"):
        print("Inicializando ChatCUCEI...")
        
        # RAG
        self.embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.chroma_client = chromadb.PersistentClient(path=chroma_db_path)
        self.collection = self.chroma_client.get_collection("profesores_cucei")
        print(f"RAG: {self.collection.count()} documentos cargados")
        
        # LLM
        self.tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model, 
            torch_dtype=torch.bfloat16, 
            device_map="auto", 
            trust_remote_code=True
        )
        self.model.eval()
        print("Modelo cargado y listo\n")
    
    def normalizar_nombre(self, texto: str) -> str:
        """Normaliza un nombre para comparacion"""
        stopwords = {'del', 'de', 'la', 'el', 'profesor', 'profesora', 'profes', 'profe', 
                    'que', 'como', 'es', 'opinas', 'recomiendas', 'sobre', 'a', 'quien',
                    'mejor', 'o'}
        
        texto = re.sub(r'[¿?¡!,.]', '', texto.lower())
        palabras = [p for p in texto.split() if p not in stopwords and len(p) > 1]
        return ' '.join(palabras).upper()
    
    def extraer_nombres(self, query: str) -> List[str]:
        """Extrae multiples nombres de una query"""
        query_limpia = re.sub(r'[¿?¡!,.]', '', query.lower())
        
        # Detectar comparaciones
        if ' o ' in query_limpia:
            partes = query_limpia.split(' o ')
            return [self.normalizar_nombre(p) for p in partes if self.normalizar_nombre(p)]
        
        return [self.normalizar_nombre(query)]
    
    def similitud_rapida(self, nombre1: str, nombre2: str) -> float:
        """Calcula similitud simple y rapida"""
        palabras1 = set(nombre1.split())
        palabras2 = set(nombre2.split())
        
        if not palabras1 or not palabras2:
            return 0.0
        
        interseccion = len(palabras1 & palabras2)
        union = len(palabras1 | palabras2)
        
        return interseccion / union if union > 0 else 0.0
    
    def buscar_mejor_match(self, nombre_query: str, candidatos: List[Dict]) -> Tuple[Optional[str], float]:
        """Encuentra el mejor match de forma eficiente"""
        if not nombre_query:
            return None, 0.0
        
        print(f"Buscando: '{nombre_query}'")
        
        mejor_profesor = None
        mejor_score = 0.0
        profesores_vistos = set()
        
        for meta in candidatos:
            prof_norm = meta['profesor_normalizado']
            
            if prof_norm in profesores_vistos:
                continue
            
            profesores_vistos.add(prof_norm)
            score = self.similitud_rapida(nombre_query, prof_norm)
            
            if score > mejor_score:
                mejor_score = score
                mejor_profesor = prof_norm
        
        if mejor_score < 0.3:
            print(f"No se encontro match confiable")
            return None, mejor_score
        
        print(f"Encontrado: {mejor_profesor} (similitud: {mejor_score:.2f})")
        return mejor_profesor, mejor_score
    
    def buscar_contexto(self, query: str, n_results: int = 25) -> Optional[Dict]:
        """Busca contexto relevante"""
        results = self.collection.query(query_texts=[query], n_results=n_results)
        
        if not results['documents'][0]:
            return None
        
        nombres = self.extraer_nombres(query)
        mejor_match, score = self.buscar_mejor_match(nombres[0], results['metadatas'][0])
        
        if not mejor_match:
            return None
        
        calificaciones = []
        tags = set()
        comentarios = []
        nombre_original = None
        
        for meta in results['metadatas'][0]:
            if meta['profesor_normalizado'] == mejor_match:
                if not nombre_original:
                    nombre_original = meta['profesor']
                
                if meta.get('calificacion', 0) > 0:
                    calificaciones.append(meta['calificacion'])
                
                if meta.get('tags'):
                    tags.update(meta['tags'].split(", "))
                
                if meta.get('comentarios'):
                    comentarios.append(meta['comentarios'])
        
        comentarios_ordenados = sorted(comentarios, key=len, reverse=True)
        
        return {
            "profesor": nombre_original,
            "calificacion_promedio": sum(calificaciones) / len(calificaciones) if calificaciones else None,
            "num_calificaciones": len(calificaciones),
            "tags": list(tags)[:5],
            "comentarios": comentarios_ordenados[:2],
            "confianza": score
        }
    
    def buscar_comparacion(self, query: str) -> Optional[Dict]:
        """Busca informacion para comparar dos profesores"""
        nombres = self.extraer_nombres(query)
        
        if len(nombres) < 2:
            return None
        
        print(f"Comparando profesores...")
        
        resultados = {}
        for nombre in nombres[:2]:
            results = self.collection.query(query_texts=[nombre], n_results=20)
            if not results['documents'][0]:
                continue
            
            mejor_match, score = self.buscar_mejor_match(nombre, results['metadatas'][0])
            if not mejor_match or score < 0.3:
                continue
            
            calificaciones = []
            tags = set()
            nombre_original = None
            
            for meta in results['metadatas'][0]:
                if meta['profesor_normalizado'] == mejor_match:
                    if not nombre_original:
                        nombre_original = meta['profesor']
                    if meta.get('calificacion', 0) > 0:
                        calificaciones.append(meta['calificacion'])
                    if meta.get('tags'):
                        tags.update(meta['tags'].split(", "))
            
            if nombre_original:
                resultados[nombre_original] = {
                    "calificacion": sum(calificaciones) / len(calificaciones) if calificaciones else None,
                    "tags": list(tags)[:3],
                    "num_evaluaciones": len(calificaciones)
                }
        
        return resultados if len(resultados) >= 2 else None
    
    def construir_prompt(self, info: Dict, query: str) -> str:
        """Construye el prompt para el modelo"""
        contexto = f"Profesor: {info['profesor']}\n"
        
        if info['calificacion_promedio']:
            contexto += f"Calificacion: {info['calificacion_promedio']:.1f}/10\n"
        
        if info['tags']:
            contexto += f"Tags: {', '.join(info['tags'][:3])}\n"
        
        if info['comentarios']:
            contexto += f"Comentario: {info['comentarios'][0][:120]}...\n"
        
        prompt = (
            f"<|system|>Eres ChatCUCEI. Responde brevemente sobre profesores.<|end|>"
            f"<|user|>{contexto}\n{query}<|end|>"
            f"<|assistant|>"
        )
        
        return prompt
    
    def generar_respuesta(self, query: str) -> str:
        """Genera respuesta usando RAG + LLM"""
        # Detectar si es comparacion
        if ' o ' in query.lower():
            comparacion = self.buscar_comparacion(query)
            if not comparacion:
                return "No se pudo encontrar informacion suficiente para comparar a esos profesores."
            
            # Respuesta directa para comparacion
            respuesta = "Comparacion de profesores:\n\n"
            for nombre, info in comparacion.items():
                respuesta += f"{nombre}:\n"
                if info['calificacion']:
                    respuesta += f"  Calificacion: {info['calificacion']:.1f}/10 ({info['num_evaluaciones']} evaluaciones)\n"
                if info['tags']:
                    respuesta += f"  Caracteristicas: {', '.join(info['tags'])}\n"
                respuesta += "\n"
            
            return respuesta.strip()
        
        # Busqueda normal
        info = self.buscar_contexto(query)
        
        if not info:
            return "No se encontro informacion sobre ese profesor en la base de datos."
        
        prompt = self.construir_prompt(info, query)
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.7,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        respuesta_completa = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        if "<|assistant|>" in respuesta_completa:
            respuesta = respuesta_completa.split("<|assistant|>")[-1].strip()
        else:
            respuesta = respuesta_completa.split(query)[-1].strip()
        
        return respuesta


if __name__ == "__main__":
    bot = ChatCUCEI()
    
    queries_prueba = [
        "¿Que opinas de Juan Carlos Corona?",
        "Como es la profesora Patricia Rosario?",
        "Quien es mejor Eloisa o Camino?"
    ]
    
    for q in queries_prueba:
        print(f"\n{'='*60}")
        print(f"Q: {q}")
        print(f"{'='*60}")
        print(bot.generar_respuesta(q))